In [ ]:
#%matplotlib notebook
%matplotlib inline
#%config InlineBackend.figure_formats=['svg']
#%config InlineBackend.figure_formats=['pdf']

import matplotlib

matplotlib.rcParams['figure.figsize'] = (12, 9)

import scipy
import scipy.stats
import math
import numpy as np
import pandas as pd

#import ipywidgets
#from ipywidgets import interact

#import sys
#sys.path.append("/Users/jdecock/git/pub/jdhp-sap/sap-cta-data-pipeline/utils/")
#import common_functions as common

## Delta psi

In [ ]:
CSV_FILE_PATH = "../xps/best.csv"

#WAVELET_LABEL = "Wavelets-n4-K-k-C1-s3-m3-kill"
#WAVELET_LABEL = "WT-ref-f3-s3"
#WAVELET_LABEL = "WT-t24-f3-s4"
#WAVELET_LABEL = "WT-t28-f3-s5"
WAVELET_LABEL = "WT-ref-s2-2-3-3"

TAILCUT_LABEL = "Tailcut-5-10-kill"

PART = 0         # 0 for gamma, 1 for protons

FAINT_BRIGHT_BORDER = 100

df = pd.read_csv(CSV_FILE_PATH)

ref = df[df.Type == 'Ref']
ref = ref[ref.Part == PART]

tc = df[df.Type == TAILCUT_LABEL]
tc = tc[tc.Part == PART]

wt = df[df.Type == WAVELET_LABEL]
wt = wt[wt.Part == PART]

tc_wt = pd.merge(tc, wt, on="Id", suffixes=('_tc', '_wt'))  #.dropna(how='any')
df = pd.merge(tc_wt, ref, on="Id")

df["delta_tc"] = np.fmod(((df['hPsi'] - df['hPsi_tc']) * 180. / np.pi), 90.)
df["delta_wt"] = np.fmod(((df['hPsi'] - df['hPsi_wt']) * 180. / np.pi), 90.)

df.delta_tc = abs(df.delta_tc)
df.delta_wt = abs(df.delta_wt)

# Plot delta psi #####################

def plot_ratio(ax, res_tuple_tc, res_tuple_wt):
    val_of_bins_tc, bins_tc, patches_tc = res_tuple_tc
    val_of_bins_wt, bins_wt, patches_wt = res_tuple_wt
    edges_of_bins = bins_tc

    # Set ratio where val_of_bins_data is not zero
    ratio = np.divide(val_of_bins_wt,
                      val_of_bins_tc,
                      where=(val_of_bins_tc != 0))

    # Compute error on ratio (null if cannot be computed)
    # This is wrong as it's made for Gaussian distributions and here we have Poisson distribution
    error = np.divide(val_of_bins_wt * np.sqrt(val_of_bins_tc) + val_of_bins_tc * np.sqrt(val_of_bins_wt),
                       np.power(val_of_bins_tc, 2),
                       where=(val_of_bins_tc != 0))

    ax.set_ylabel('Ratio (WT/TC)', fontsize=20)
    ax.axhline(y=1, linewidth=2, linestyle='--', color='gray', alpha=0.5)

    bincenter = 0.5 * (edges_of_bins[1:] + edges_of_bins[:-1])
    ax.errorbar(bincenter, ratio, yerr=error, fmt='o', color='k', elinewidth=3, capsize=4, capthick=3, linewidth=6)
    ax.plot(bincenter, ratio, 'ok', linewidth=6)

################################

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize=(16, 9))

#BINS_START, BINS_STOP, BINS_STEP = 0., 99., 9.
#BINS_START, BINS_STOP, BINS_STEP = 0., 99., 9.

#BINS = np.arange(BINS_START, BINS_STOP, BINS_STEP)
#BINS = 30
#BINS = np.linspace(BINS_START, BINS_STOP, 30)
#BINS = [3.53056780e-03, 5.99515231e+00, 1.19867741e+01, 1.79783958e+01, 2.39700176e+01, 2.99616393e+01,
#        3.59532610e+01, 4.19448828e+01, 4.79365045e+01, 5.39281263e+01, 5.99197480e+01, 6.59113698e+01,
#        7.19029915e+01, 7.78946133e+01, 8.38862350e+01, 8.98778568e+01]
BINS = 15

################################

NPE_MIN, NPE_MAX = 50, FAINT_BRIGHT_BORDER
res_tuple_tc1 = ax1.hist(df[df.peSum_tc > NPE_MIN][df.peSum_tc <= NPE_MAX]['delta_tc'],
                         label="TC num={}".format(len(df[df.peSum_tc > NPE_MIN][df.peSum_tc <= NPE_MAX])),
                         bins=BINS,
                         linewidth=2, alpha=.5, color='blue', histtype="step")

res_tuple_wt1 = ax1.hist(df[df.peSum_wt > NPE_MIN][df.peSum_wt <= NPE_MAX]['delta_wt'],
                         label="WT num={}".format(len(df[df.peSum_wt > NPE_MIN][df.peSum_wt <= NPE_MAX])),
                         bins=BINS,
                         linewidth=2, alpha=.5, color='red',  histtype="step")

ax1.set_title("{} to {} NPE".format(NPE_MIN, NPE_MAX), fontsize=20)
ax1.set_xlabel("delta_psi", fontsize=20)
ax1.set_ylabel("counts", fontsize=20)
ax1.legend(prop={'size': 18}, loc='best', fancybox=True, framealpha=0.5)

plot_ratio(ax3, res_tuple_tc1, res_tuple_wt1)

################################

NPE_MIN, NPE_MAX = FAINT_BRIGHT_BORDER, 2000
res_tuple_tc2 = ax2.hist(df[df.peSum_tc > NPE_MIN][df.peSum_tc <= NPE_MAX]['delta_tc'],
                         label="TC num={}".format(len(df[df.peSum_tc > NPE_MIN][df.peSum_tc <= NPE_MAX])),
                         bins=BINS,
                         linewidth=2, alpha=.5, color='blue', histtype="step")

res_tuple_wt2 = ax2.hist(df[df.peSum_wt > NPE_MIN][df.peSum_wt <= NPE_MAX]['delta_wt'],
                         label="WT num={}".format(len(df[df.peSum_wt > NPE_MIN][df.peSum_wt <= NPE_MAX])),
                         bins=BINS,
                         linewidth=2, alpha=.5, color='red',  histtype="step")

ax2.set_title("{} to {} NPE".format(NPE_MIN, NPE_MAX), fontsize=20)
ax2.set_xlabel("delta_psi", fontsize=20)
ax2.set_ylabel("counts", fontsize=20)
ax2.legend(prop={'size': 18}, loc='best', fancybox=True, framealpha=0.5)

plot_ratio(ax4, res_tuple_tc2, res_tuple_wt2)

#ax.set_yscale('log')

## Delta psi from script

Rem: this is a cross check, you can ignore it...

In [ ]:
#%run ./utils/plot_delta_psi.py --notebook --metric hillas2_delta_psi_norm2 xps/2017_02_23_sapcta/run_8/score_gamma_all_wavelets_mrfilter_kill.json xps/2017_02_22_sapcta/run_1/score_gamma_all_tailcut_kill.json